###### 2020-10-15 목요일 / 2020-10-16 금요일
 
# Pandas : pd.DataFrame 실습

   > - groupby를 적용하면 쉽게 해결되는 문제들이지만...
   > - 쓰지않고 풀어보세요. 그러면 실력이 늘거에요

##### 어제 제공된 3개의 파일을 활용한 [실습]
    - ratings.dat
    - movies.dat
    - users.dat

##### 1. 각 파일을 pd.read_csv() 로 읽어들인다
    - 헤더 및 구분자 확인요망

##### 2. 데이터 확인 및 구조분석과 각 데이터 프레임에 헤더를 추가
    ratings.dat
        - userId / movieId / rating / timestamp
    movies.data
        - movieId / title / genres
    users.data (총 5개 행 중에서 뒤의 2개행은 제거)
        - userId / gender / age

##### 3. 공통 컬럼을 이용한 데이터 병합

##### 4. user 평점 갯수 순위 확인

##### 5. 각 영화의 성별 평균 평점

##### 6. 평점이 220개 이상인 영화만 필터링

##### 7. 평점이 200개 이상인 영화들의 성별 평균 평점

##### 8. 여성이 가장 좋아하는 영화 순위

##### 9. 남녀간의 평균 평점 차이

##### 10. 평점의 표준편차가 큰 영화

##### (+추가) Comedy영화 중 가장 평점이 낮은 영화의 제목

##### (+추가) 평균 평점이 가장 높은 영화의 제목(동률이 있을 경우 모두 출력)

##### (+추가) 각 영화별 평균 평점



#### 5번 ,6번, 9번은 groupby()와 pivot함수를 적용하여 풀었습니다.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

##### 1. 각 파일을 pd.read_csv() 로 읽어들인다
- 헤더 및 구분자 확인요망

In [2]:
ratings_data = pd.read_csv('./실습데이터/ratings.dat', sep='::', header=None)
movies_data = pd.read_csv('./실습데이터/movies.dat', sep='::', header=None)
users_data = pd.read_csv('./실습데이터/users.dat', sep='::', header=None)

C:\Users\lan41\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
C:\Users\lan41\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
C:\Users\lan41\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from th

In [3]:
display(ratings_data.head(10))
display(movies_data.head(10))
display(users_data.head(10))

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


,0,1,2
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


,0,1,2,3,4
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
5,6,F,50,9,55117
6,7,M,35,1,06810
7,8,M,25,12,11413
8,9,M,25,17,61614
9,10,F,35,1,95370


##### 2. 데이터 확인 및 구조분석과 각 데이터 프레임에 헤더를 추가
   - ratings_data : userId movieId rating timestamp
   - movies_data : movieId title genres
   - users_data : userId gender age (뒤에 두 칼럼은 제거한다)

In [4]:
ratings_data.columns = ['user_id', 'movie_id', 'rating', 'timestamp']
movies_data.columns = ['movie_id', 'title', 'genres']
users_data.columns  = ['user_id', 'gender', 'age', 'drop1', 'drop2']

del users_data['drop1']
del users_data['drop2']

display(ratings_data.head())
display(movies_data.head())
display(users_data.head())

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


,user_id,gender,age
0,1,F,1
1,2,M,56
2,3,M,25
3,4,M,45
4,5,M,25


##### 3. 공통 컬럼을 이용한 데이터 병합

In [5]:
two_df = pd.merge(ratings_data, movies_data, on='movie_id', how='left')
all_df = pd.merge(two_df, users_data, on='user_id', how='left')

display(all_df.head(10))
display(all_df.tail(10))

,user_id,movie_id,rating,timestamp,title,genres,gender,age
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1
5,1,1197,3,978302268,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance,F,1
6,1,1287,5,978302039,Ben-Hur (1959),Action|Adventure|Drama,F,1
7,1,2804,5,978300719,"Christmas Story, A (1983)",Comedy|Drama,F,1
8,1,594,4,978302268,Snow White and the Seven Dwarfs (1937),Animation|Children's|Musical,F,1
9,1,919,4,978301368,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical,F,1


,user_id,movie_id,rating,timestamp,title,genres,gender,age
1000199,6040,2022,5,956716207,"Last Temptation of Christ, The (1988)",Drama,M,25
1000200,6040,2028,5,956704519,Saving Private Ryan (1998),Action|Drama|War,M,25
1000201,6040,1080,4,957717322,Monty Python's Life of Brian (1979),Comedy,M,25
1000202,6040,1089,4,956704996,Reservoir Dogs (1992),Crime|Thriller,M,25
1000203,6040,1090,3,956715518,Platoon (1986),Drama|War,M,25
1000204,6040,1091,1,956716541,Weekend at Bernie's (1989),Comedy,M,25
1000205,6040,1094,5,956704887,"Crying Game, The (1992)",Drama|Romance|War,M,25
1000206,6040,562,5,956704746,Welcome to the Dollhouse (1995),Comedy|Drama,M,25
1000207,6040,1096,4,956715648,Sophie's Choice (1982),Drama,M,25
1000208,6040,1097,4,956715569,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi,M,25


##### 4. user 평점 갯수 순위 확인

In [6]:
temp_data = pd.DataFrame({
    'user_id_count' : all_df['user_id'].value_counts()
    })

temp_data.index.name = 'user_id'
temp_data = temp_data.reset_index()
temp_data['user_id_count_rank'] = temp_data['user_id_count'].rank(ascending=False).astype(int)
temp_data

all_df2 = pd.merge(all_df, temp_data, how='left', on='user_id')
display(all_df2.head(10))

,user_id,movie_id,rating,timestamp,title,genres,gender,age,user_id_count,user_id_count_rank
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,53,4149
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,53,4149
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,53,4149
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,53,4149
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,53,4149
5,1,1197,3,978302268,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance,F,1,53,4149
6,1,1287,5,978302039,Ben-Hur (1959),Action|Adventure|Drama,F,1,53,4149
7,1,2804,5,978300719,"Christmas Story, A (1983)",Comedy|Drama,F,1,53,4149
8,1,594,4,978302268,Snow White and the Seven Dwarfs (1937),Animation|Children's|Musical,F,1,53,4149
9,1,919,4,978301368,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical,F,1,53,4149


##### 5. 각 영화의 성별 평균 평점 (group / pivot 사용)

In [32]:
pivot_df = all_df.pivot_table('rating', 'title', 'gender', aggfunc='mean')
pivot_df.head(10)

gender,F,M
title,,
"$1,000,000 Duck (1971)",3.375000,2.761905
'Night Mother (1986),3.388889,3.352941
'Til There Was You (1997),2.675676,2.733333
"'burbs, The (1989)",2.793478,2.962085
...And Justice for All (1979),3.828571,3.689024
1-900 (1994),2.000000,3.000000
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215


##### 6. 평점이 220개 이상인 영화만 필터링  (group / pivot 사용)

In [57]:
all_df.head()

,user_id,movie_id,rating,timestamp,title,genres,gender,age
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1


In [52]:
# count_movie = all_df.groupby('title').apply(np.size)
# up220_movie = np.unique(count_movie[count_movie >= 220].index)
# up220_index = list(map(lambda x : x in up220_movie, all_df['title'])) >> 너무느려..

In [76]:
movie_count = all_df['title'].value_counts()
movie_count_df = pd.DataFrame( { 'movie_count' : movie_count })
movie_count_df.index.name = 'title'
movie_count_df = movie_count_df.reset_index()

all_df_cnt = pd.merge(all_df, movie_count_df, on='title', how='left')
up220_df = all_df_cnt[ all_df_cnt['movie_count'] >= 220 ]

# 평점수가 220개인 영화만 출력
up220_df.head(10)

,user_id,movie_id,rating,timestamp,title,genres,gender,age,movie_count
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,1725
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,525
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,636
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,1315
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,1703
5,1,1197,3,978302268,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance,F,1,2318
6,1,1287,5,978302039,Ben-Hur (1959),Action|Adventure|Drama,F,1,704
7,1,2804,5,978300719,"Christmas Story, A (1983)",Comedy|Drama,F,1,1352
8,1,594,4,978302268,Snow White and the Seven Dwarfs (1937),Animation|Children's|Musical,F,1,763
9,1,919,4,978301368,"Wizard of Oz, The (1939)",Adventure|Children's|Drama|Musical,F,1,1718


##### 7. 평점이 200개 이상인 영화들의 성별 평균 평점

In [7]:
# 영화들이 가지는 행의 개수를 세어 데이터프레임의 칼럼으로 만들고 all_df3에 merge하자
temp_data = pd.DataFrame( {'movie_count' : all_df2['movie_id'].value_counts()} )
temp_data.index.name = 'movie_id'
temp_data = temp_data.reset_index()
all_df3 = pd.merge(all_df2, temp_data, on='movie_id', how='left')
all_df3.head()

,user_id,movie_id,rating,timestamp,title,genres,gender,age,user_id_count,user_id_count_rank,movie_count
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,53,4149,1725
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,53,4149,525
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,53,4149,636
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,53,4149,1315
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,53,4149,1703


In [8]:
# 영화의 평점이 200개 이상인 데이터프레임 up_200_movie를 만들자
up_200_movie = all_df3[all_df3.movie_count >= 200]
display(up_200_movie.head())
display(up_200_movie.tail())

,user_id,movie_id,rating,timestamp,title,genres,gender,age,user_id_count,user_id_count_rank,movie_count
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,53,4149,1725
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,53,4149,525
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,53,4149,636
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,53,4149,1315
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,53,4149,1703


,user_id,movie_id,rating,timestamp,title,genres,gender,age,user_id_count,user_id_count_rank,movie_count
1000204,6040,1091,1,956716541,Weekend at Bernie's (1989),Comedy,M,25,341,778,373
1000205,6040,1094,5,956704887,"Crying Game, The (1992)",Drama|Romance|War,M,25,341,778,1229
1000206,6040,562,5,956704746,Welcome to the Dollhouse (1995),Comedy|Drama,M,25,341,778,478
1000207,6040,1096,4,956715648,Sophie's Choice (1982),Drama,M,25,341,778,344
1000208,6040,1097,4,956715569,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi,M,25,341,778,2269


In [9]:
# 영화 평점이 200개 이상인 영화들의 성별별로 평점을 구해보자

# 남
up_200_namja_rating = up_200_movie[up_200_movie['gender'] == 'M'].rating.mean()

# 여
up_200_yeoja_rating = up_200_movie[up_200_movie['gender'] == 'F'].rating.mean()


print('영화리뷰개수가 200개 이상인 영화들의 남자의 평균평점 :', up_200_namja_rating)
print('영화리뷰개수가 200개 이상인 영화들의 여자의 평균평점 :', up_200_yeoja_rating)

영화리뷰개수가 200개 이상인 영화들의 남자의 평균평점 : 3.650922994518683
영화리뷰개수가 200개 이상인 영화들의 여자의 평균평점 : 3.688517650906957


##### 8. 여성이 가장 좋아하는 영화 순위


In [10]:
# 여성의 데이터만 추출한다
only_female = all_df3[all_df3.gender == 'F']
only_female.head()

,user_id,movie_id,rating,timestamp,title,genres,gender,age,user_id_count,user_id_count_rank,movie_count
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,53,4149,1725
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,53,4149,525
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,53,4149,636
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,53,4149,1315
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,53,4149,1703


In [11]:
# 유니크한 movie_id를 추춯한다.
fac_movie_id = np.unique(only_female['movie_id'])

In [12]:
# fac_movie_id을 이용해서 영화별 평점을 구하자
average_movie_rating = []
for movie_id in fac_movie_id:
    average_movie_rating.append( only_female[only_female['movie_id'] == movie_id].rating.mean() )

In [13]:
average_movie_rating_df = pd.DataFrame({
    'movie_id'              : fac_movie_id,
    'average_movie_rating'  : average_movie_rating
})

average_movie_rating_df.head()

,movie_id,average_movie_rating
0,1,4.187817
1,2,3.278409
2,3,3.073529
3,4,2.976471
4,5,3.212963


In [14]:
# all_df3에서 영화의 이름만을 가져와 average_movie_rating_df와 merge하자

# all_df3 에서 'movie_id', 'title'칼럼을 가져와 유니크한 행을 가지는 데이터프레임으로 만든다
temp_df = all_df3[ ['movie_id', 'title'] ].drop_duplicates(['movie_id', 'title'], keep='first')
average_movie_rating_df2 = pd.merge(average_movie_rating_df, temp_df, on='movie_id', how='left')

# average_movie_rating_df2에서 영화 평균 평점(average_movie_rating)을 내림차순으로 정렬한다.
average_movie_rating_df3 = average_movie_rating_df2.sort_values(by='average_movie_rating', ascending=False)

# 평점이 높은 상위 10개의 영화를 출력하자
average_movie_rating_df3.head(10)

,movie_id,average_movie_rating,title
50,53,5.0,Lamerica (1994)
3356,3817,5.0,"Other Side of Sunday, The (S�ndagsengler) (1996)"
1227,1420,5.0,Message to Love: The Isle of Wight Festival (1...
2007,2332,5.0,Belly (1998)
1829,2129,5.0,"Saltmen of Tibet, The (1997)"
2889,3292,5.0,"Big Combo, The (1955)"
2645,3012,5.0,Battling Butler (1926)
1251,1450,5.0,Prisoner of the Mountains (Kavkazsky Plennik) ...
2118,2444,5.0,24 7: Twenty Four Seven (1997)
2616,2981,5.0,"Brother, Can You Spare a Dime? (1975)"


##### 9. 남녀간의 평균 평점 차이

In [15]:
male_df = all_df3[all_df3['gender'] == 'M']
female_df = all_df3[all_df3['gender'] == 'F']

cha_e = male_df.rating.mean() - female_df.rating.mean()
print('남녀간의 평균 평점 차이 :', np.abs(cha_e))

남녀간의 평균 평점 차이 : 0.05148748291259997


##### 10. 평점의 표준편차가 큰 영화 (group / pivot 사용)

In [136]:
rating_std = all_df[['title', 'rating']].groupby('title').agg(np.std)
rating_std.columns = ['rating_std']

sort_rating_std = rating_std[['rating_std']].sort_values(by='rating_std', ascending=False)

# 표준편차가 가장 큰 영화
sort_rating_std.iloc[[0], :]

,rating_std
title,
Foreign Student (1994),2.828427


##### (+추가) Comedy영화 중 가장 평점이 낮은 영화의 제목

In [16]:
# Comedy 영화만 추출한다
comedy_df = all_df3[ all_df3['genres'] == 'Comedy' ]
comedy_df.head(10)

,user_id,movie_id,rating,timestamp,title,genres,gender,age,user_id_count,user_id_count_rank,movie_count
13,1,2918,4,978302124,Ferris Bueller's Day Off (1986),Comedy,F,1,53,4149,1473
15,1,2791,4,978302188,Airplane! (1980),Comedy,F,1,53,4149,1731
20,1,2321,3,978302205,Pleasantville (1998),Comedy,F,1,53,4149,1158
55,2,1537,4,978299620,Shall We Dance? (Shall We Dansu?) (1996),Comedy,M,56,129,2371,350
132,2,2359,3,978299666,Waking Ned Devine (1998),Comedy,M,56,129,2371,703
141,2,3809,3,978299712,What About Bob? (1991),Comedy,M,56,129,2371,817
150,2,2321,3,978299666,Pleasantville (1998),Comedy,M,56,129,2371,1158
182,3,3421,4,978298147,Animal House (1978),Comedy,M,25,51,4226,1207
183,3,1641,2,978298430,"Full Monty, The (1997)",Comedy,M,25,51,4226,1199
185,3,1394,4,978298147,Raising Arizona (1987),Comedy,M,25,51,4226,1434


In [17]:
# comedy 영화별 평균 평점을 구한다
unique_movie_id = np.unique(comedy_df['movie_id'])

comedy_rating = []
for movie_id in unique_movie_id:
    temp = comedy_df[comedy_df['movie_id'] == movie_id].rating.mean()
    comedy_rating.append(temp)
    
comedy_rating[0:10]

[3.0067567567567566,
 2.480719794344473,
 2.8214285714285716,
 3.640371229698376,
 2.855855855855856,
 2.062937062937063,
 3.6551724137931036,
 2.9427083333333335,
 3.923076923076923,
 3.869565217391304]

In [18]:
# comedy 영화별 평균영화평점 데이터 프레임 만들기

comedy_average_df = pd.DataFrame({
    'movie_id'      : unique_movie_id,
    'comedy_average_rating' : comedy_rating
})
comedy_average_df.head(10)

,movie_id,comedy_average_rating
0,5,3.006757
1,19,2.480720
2,38,2.821429
3,52,3.640371
4,63,2.855856
5,65,2.062937
6,69,3.655172
7,88,2.942708
8,96,3.923077
9,101,3.869565


In [19]:
# comedy_df에서 영화의 이름만을 가져와 comedy_average_df merge하자

# comedy_df에서 'movie_id', 'title'칼럼을 가져와 유니크한 행을 가지는 데이터프레임으로 만든다
temp_df = comedy_df[ ['movie_id', 'title'] ].drop_duplicates(['movie_id', 'title'], keep='first')
comedy_average_df2 = pd.merge(comedy_average_df, temp_df, on='movie_id', how='left')

# comedy_average_df2에서 영화 평균 평점(comedy_average_rating)을 오름차순으로 정렬한다.
comedy_average_df3 = comedy_average_df2.sort_values(by='comedy_average_rating')


# 평점이 가장 낮은(평점이 1인 영화들을 확인)
comedy_average_df3[ comedy_average_df3['comedy_average_rating'] == 1 ]


,movie_id,comedy_average_rating,title
156,1311,1.0,Santa with Muscles (1996)
419,3460,1.0,Hillbillys in a Haunted House (1967)
90,641,1.0,"Little Indian, Big City (Un indien dans la vil..."
94,655,1.0,Mutters Courage (1995)
382,3228,1.0,Wirey Spindell (1999)
372,3123,1.0,Spring Fever USA (a.k.a. Lauderdale) (1989)


##### (+추가) 평균 평점이 가장 높은 영화의 제목(동률이 있을 경우 모두 출력)

In [20]:
# 유니크한 movie_id를 만든다
unique_movie_id = np.unique(all_df3['movie_id'])

In [21]:
average_rating = []

# 유니크한 movie_id를 이용해 영화별 평균 평점 average_rating을 구한다.
for movie_id in unique_movie_id:
    average_rating.append( all_df3[ all_df3['movie_id'] == movie_id ].rating.mean() )

In [22]:
# unique_movie_id와 average_rating을 이용해 데이터프레임을 만든다.

average_movie_df = pd.DataFrame({
    'movie_id'              : unique_movie_id,
    'average_movie_rating'  : average_rating
})
average_movie_df.head()

,movie_id,average_movie_rating
0,1,4.146846
1,2,3.201141
2,3,3.016736
3,4,2.729412
4,5,3.006757


In [23]:
# all_df3에서 영화의 이름만을 가져와 average_movie_df와 merge하자

# all_df3에서 'movie_id', 'title'칼럼을 가져와 유니크한 행을 가지는 데이터프레임으로 만든다
temp_df = all_df3[ ['movie_id', 'title'] ].drop_duplicates(['movie_id', 'title'], keep='first')
average_movie_df2 = pd.merge(average_movie_df, temp_df, on='movie_id', how='left')

# comedy_average_df2에서 영화 평균 평점(average_movie_rating)을 내림차순으로 정렬한다.
average_movie_df3 = average_movie_df2.sort_values(by='average_movie_rating', ascending=False)

# 영화 평균 평점이 가장 높은 영화(평점 5인 영화)를 출력한다
average_movie_df3[average_movie_df3[ 'average_movie_rating' ] == 5 ]


,movie_id,average_movie_rating,title
926,989,5.0,Schlafes Bruder (Brother of Sleep) (1995)
3635,3881,5.0,Bittersweet Motel (2000)
1652,1830,5.0,Follow the Bitch (1998)
3152,3382,5.0,Song of Freedom (1936)
744,787,5.0,"Gate of Heavenly Peace, The (1995)"
3054,3280,5.0,"Baby, The (1973)"
3367,3607,5.0,One Little Indian (1973)
3010,3233,5.0,Smashing Time (1967)
2955,3172,5.0,Ulysses (Ulisse) (1954)
3414,3656,5.0,Lured (1947)


##### (+추가) 각 영화별 평균 평점

In [24]:
# `(+추가) 평균 평점이 가장 높은 영화의 제목(동률이 있을 경우 모두 출력)` 문제 출면서 구했음

average_movie_df2.head()

,movie_id,average_movie_rating,title
0,1,4.146846,Toy Story (1995)
1,2,3.201141,Jumanji (1995)
2,3,3.016736,Grumpier Old Men (1995)
3,4,2.729412,Waiting to Exhale (1995)
4,5,3.006757,Father of the Bride Part II (1995)
